# pymysql
- 파이썬에서 MySQL, MariaDB DBMS와 연동하는 다양한 함수를 제공하는 모듈
- Python [DB API 2.0](http://www.python.org/dev/peps/pep-0249) 표준을 따른다.
- https://github.com/PyMySQL/PyMySQL/
- https://pymysql.readthedocs.io/en/latest/

# 설치
- 조건
    - python version 3.6 이상
    - mysql version 5.6 이상
- 설치
    - `pip install pymysql`
    - `conda install -c conda-forge pymysql`

In [ ]:
!pip install pymysql

# 기본 작성 절차

1. Database 연결
    ```python
       connection =  pymysql.connect(host="DBMS 서버 ip", 
                                     port="port번호", 
                                     user="계정명", 
                                     password="비밀번호", 
                                     db="연결할데이터베이스이름", 
                                     charset='utf8')
    ```
    - port 번호 기본값: 3306
2. Connection을 이용해 Cursor 생성
    - Cursor: 연결된 Database에 sql문을 전송하고 select결과 조회 기능을 제공하는 객체
    ```python
        cursor = connection.cursor()
    ```
3. Cusror를 이용해 SQL문 전송
    ```python
        cursor.execute("sql문")
    ```
4. 연결 닫기
    - cursor, connection 연결을 닫는다.
    - with문을 이용할 수 있다. 
    ```python
    cursor.close()
    connection.close()
    ```

# 예제

## 테이블 생성

In [6]:
CREATE_SQL = """
create table member (
 id   int  auto_increment  primary key,
 name varchar(30)  not null,
 email varchar(100) not null unique,
 tall  decimal(5,2), 
 birthday date,
 created_at datetime
)
"""

In [7]:
import pymysql
## 1. DB와 연결
conn = pymysql.connect(host="127.0.0.1",  ## DB서버 ip
                       port=3306,         ## 연결할 DBMS port 번호
                       user="scott",      # username
                       password="tiger",  # password
                       db='testdb'        # 연결할 database이름.
                      )
print(type(conn))

## 2. Connection 객체를 이용해 Cursor(SQL작업을 관리)를 생성.
cursor = conn.cursor()
print(type(cursor))

## 3. SQL 문을 실행 - cursor의 execute()메소드 이용
cnt = cursor.execute(CREATE_SQL)
print("cnt:", cnt)  # sql 적용을 받은 데이터수를 반환.

## Database와 연결 닫기 (반드시 해야함.) ==> cursor close, conn close
cursor.close()
conn.close()   

<class 'pymysql.connections.Connection'>
<class 'pymysql.cursors.Cursor'>
cnt: 0


## DML
### insert

In [12]:
# sql문 -> mysql 기준.
insertSQL = "insert into member (name, email, tall, birthday, created_at) \
values ('이순신', 'lee1@a.com', 180.23, '1990-10-02', now())"

# 1. DB 연결
with pymysql.connect(host="127.0.0.1", port=3306, 
                     user='scott',password='tiger', 
                     db="testdb") as conn:
    # 2. Cursor 생성
    with conn.cursor() as cursor:
        # 3. sql문 실행 
        cnt = cursor.execute(insertSQL)
        print('insert 개수:', cnt)
        conn.commit()   
        ### pymysql 은 수동 commit이 default임. 그래서 insert/update/delete
        #### 후에는 commit을 해야 연결된 DB에 적용된다.
    

# 4. 연결닫기 -> Cursor/DB ==> with block 나오면서 자동으로 끊긴다.

insert 개수: 1


### Parameterized Query
- Parameterized Query
    - SQL 문에서 컬럼 값이 들어가는 자리에 값대신 `%s` placeholder를 사용한뒤 execute()에서 placeholder에 넣을 값을 list나 tuple로 제공한다.
    - query문을 쉽게 작성할 수 있는 장점이 있다.        

In [13]:
sql = "insert into member (name, email, tall, birthday, created_at) \
values (%s, %s, %s, %s, %s)"

In [14]:
import datetime
with pymysql.connect(host="127.0.0.1", port=3306, 
                     user='scott', password='tiger', db='testdb') as conn:
    with conn.cursor() as cursor:
        cnt = cursor.execute(sql, ("강감찬", "k@a.com", 190.2, 
                                   datetime.date(2000, 1, 2), 
                                   datetime.datetime.now()))
        conn.commit()

### Parameterized Query를 이용해 여러개 행 insert

#### for문 사용

In [15]:
from datetime import date, datetime

datas = [
    ['name1', 'abc1@abc.com', 165, date(2000,1,12), datetime.now()],
    ['name2', 'def1@abc.com', 175, date(1995,12,20), datetime.now()],
    ['name3', 'ghi1@abc.com', 185, date(1988, 7, 21), datetime.now()]
]

In [17]:
sql = "insert into member (name, email, tall, birthday, created_at) \
values (%s, %s, %s, %s, %s)"
with pymysql.connect(host="127.0.0.1", port=3306, 
                     user='scott', password='tiger', db='testdb') as conn:
    with conn.cursor() as cursor:
        
        
        cnt = 0
        for data in datas:
            i = cursor.execute(sql, data)
            cnt += i
        conn.commit()
        
print(f"{cnt} 행 추가")

3 행 추가


#### executemany() 사용
- insert할 값들을 가진 리스트를 넣어 한번에 여러 행을 insert한다.

In [19]:
datas = [
    ['이름1', 'abc2@a.com', 165, date(2000,1,12), datetime.now()],
    ['이름2', 'def2@a.com', 175, date(1995,12,20), datetime.now()],
    ['이름1', 'ghi2@a.com', 185, date(1988, 7, 21), datetime.now()]
]

In [20]:
sql = "insert into member (name, email, tall, birthday, created_at) \
values (%s, %s, %s, %s, %s)"
with pymysql.connect(host="127.0.0.1", port=3306, 
                     user='scott', password='tiger', db='testdb') as conn:
    with conn.cursor() as cursor:
        cnt = cursor.executemany(sql, datas)
        print(cnt)
        conn.commit()

3


### update/delete
- 코딩 절차는 insert 와 동일

## select (DQL - Data Query Language)
- 조회결과 조회
    - cursor.execute("select문") 실행 후 cursor의 결과 조회 메소드(fetch메소드)를 이용해 결과를 받는다.
- fetch메소드
    - **fetchall()**
        - 조회한 모든 행을을 반환
    - **fetchmany(size=개수)**
        - 지정한 size개수 만큼 반환
    - **fetchone()**
        - 조회결과 중 첫번째 행만 반환
        - 주로 pk 동등 조건으로 조회한 경우 사용

### fetchall()

### fetchone()

### fetchmany()

### cursor 는 iterable 타입
- for문에 select 실행한 cursor를 사용하면 조회결과를 한 행씩 조회할 수 있다.

# 함수 구현

보통 테이블과 관련해서 CRUD를 처리하는 함수 또는 클래스를 정의해서 사용한다.  
다음은 Member 테이블의 CRUD를 처리하는 함수를 작성한다.

1. name, email, tall, birthday를 매개변수로 받아서 insert하는 함수. (id는 자동증가, created_at은 실행시점의 일시가 insert되도록 한다.)
2. id, name, email, tall, birthday를 매개변수로 받아서 id의 member의 나머지 정보를 update하는 함수. (created_at은 update하지 않는다.)
3. id를 매개변수로 받아서 그 member 를 삭제하는 함수.
4. id를 매개변수로 받아서 그 id의 회원 정보를 조회하여 반환하는 함수.
5. 전체 회원정보를 조회하는 함수


- insert, update, delete 는 적용된 행의 개수를 반환한다.
- select 처리 함수는 조회결과를 반환한다.
